# LOGISTIC REGRESSION

La regresión logística es un modelo de clasificación lineal que intenta aprender la relación entre variables independientes (en este caso, el contenido textual de los statements) y una variable binaria (si el statement es falso o verdadero).

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix

In [ ]:
# Cargar los datos

df = pd.read_csv('../../../../data/processed/train_preprocess_v1.csv')  # Ajusta la ruta si es necesario
df.head

<bound method NDFrame.head of                id  label                                          statement  \
0     81f884c64a7      1  china is in the south china sea and (building)...   
1     30c2723a188      0  with the resources it takes to execute just ov...   
2     6936b216e5d      0  the (wisconsin) governor has proposed tax give...   
3     b5cd9195738      1  says her representation of an ex-boyfriend who...   
4     84f8dac7737      0  at protests in wisconsin against proposed coll...   
...           ...    ...                                                ...   
8945  44edff2b865      1  if rhode island does a hybrid [retirement] pla...   
8946  4a63b5f9c16      1  the new health care law will force seniors int...   
8947  7c57fa8e81c      0  the health insurance plan that (members of con...   
8948  2375e3cf4b7      1  no one in american history has moved from a ju...   
8949  5ae9b14e6e5      0  says the army is spending $7 million to sponso...   

                     

### Procesamiento combinado con columnas ColumnTransformer

ColumnTransformer permite aplicar distintos preprocesamientos a diferentes tipos de columnas:
- statement: se vectoriza usando TF-IDF.
- Las columnas categóricas se transforman con OneHotEncoder.

handle_unknown='ignore': ignora categorías no vistas durante entrenamiento.

### Pipeline de entrenamiento

- Unifica el preprocesamiento y el modelo en un flujo automático.
- max_iter=1000: asegura convergencia de la regresión logística.

### Definición de hiperparámetros para buscar

- C: controla la regularización (valores bajos = más regularización).
- penalty: tipo de regularización (l1 = Lasso, l2 = Ridge).
- solver: liblinear es compatible con l1.

### GridSearchCV para búsqueda de hiperparámetros
- cv=5: validación cruzada en 5 particiones.
- scoring='accuracy': métrica usada para elegir el mejor modelo.
- n_jobs=-1: usa todos los núcleos de CPU.
- verbose=2: muestra progreso del entrenamiento.

In [7]:
# Seleccionar columnas y manejar valores faltantes
df = df[['statement', 'subject', 'speaker', 'party_affiliation', 'speaker_job', 'label']].fillna("unknown")

# Separar features y labels
X = df[['statement', 'subject', 'speaker', 'party_affiliation', 'speaker_job']]
y = df['label']

# Preprocesamiento combinado
preprocessor = ColumnTransformer(transformers=[
    ('text', TfidfVectorizer(stop_words='english', max_features=5000), 'statement'),
    ('cat', OneHotEncoder(handle_unknown='ignore'), ['subject', 'speaker', 'party_affiliation', 'speaker_job'])
])

# Pipeline: preprocesamiento + modelo base
pipeline = Pipeline([
    ('preprocess', preprocessor),
    ('clf', LogisticRegression(max_iter=1000))
])

# Parámetros para tuning
param_grid = {
    'clf__C': [0.01, 0.1, 1, 10],
    'clf__penalty': ['l1', 'l2'],
    'clf__solver': ['liblinear']  # necesario para que funcione con l1
}

# GridSearchCV con validación cruzada de 5 folds
grid_search = GridSearchCV(pipeline, param_grid, cv=5, scoring='accuracy', n_jobs=-1, verbose=2)

# División train/test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Entrenar
grid_search.fit(X_train, y_train)

# Mejor modelo y resultados
best_model = grid_search.best_estimator_
print("Mejores parámetros encontrados:", grid_search.best_params_)

# Evaluación
y_pred = best_model.predict(X_test)
print("\nMatriz de confusión:\n", confusion_matrix(y_test, y_pred))
print("\nReporte de clasificación:\n", classification_report(y_test, y_pred))


Fitting 5 folds for each of 8 candidates, totalling 40 fits
Mejores parámetros encontrados: {'clf__C': 1, 'clf__penalty': 'l1', 'clf__solver': 'liblinear'}

Matriz de confusión:
 [[ 168  443]
 [ 178 1001]]

Reporte de clasificación:
               precision    recall  f1-score   support

           0       0.49      0.27      0.35       611
           1       0.69      0.85      0.76      1179

    accuracy                           0.65      1790
   macro avg       0.59      0.56      0.56      1790
weighted avg       0.62      0.65      0.62      1790



## Conclusión

El modelo es claramente mejor reconociendo afirmaciones verdaderas que falsas.
- Tiene un recall alto del 85% en clase 1 (verdadero): es útil si el objetivo es identificar con seguridad lo que es cierto.
- Pero solo detecta correctamente el 27% de las afirmaciones falsas, lo que genera una alta tasa de falsos negativos: muchas falsedades pasan como verdaderas.

El desequilibrio en la capacidad predictiva entre clases indica un sesgo hacia la clase 1. Esto puede deberse a:
- Desbalance en los datos (hay más verdaderos que falsos).
- El lenguaje en afirmaciones falsas puede ser más variado, irónico o ambiguo, lo que el modelo no capta bien.
- Limitación de un modelo lineal como la regresión logística para capturar relaciones complejas o semánticas.

Las variables categóricas como speaker, subject y party_affiliation ayudan, pero no resuelven del todo el problema.
- Su inclusión mejora el rendimiento respecto a usar solo texto, pero aún hay espacio para mejoras sustanciales.

## Generación de archivo de predicciones

In [ ]:
import joblib
import datetime
import pandas as pd

# Cargar el test

test_df = pd.read_csv('../../../../data/processed/test_preprocess_v1.csv')

# Asegúrate de que las columnas coincidan con las usadas para entrenar
test_df = test_df[['statement', 'subject', 'speaker', 'party_affiliation', 'speaker_job']].fillna("unknown")

# Usar el mejor modelo obtenido anteriormente para predecir
y_pred_test = best_model.predict(test_df)

# Cargar archivo de ejemplo de envío para respetar el formato
submission = pd.read_csv("sample_submission.csv")

# Insertar las predicciones en el campo correcto
submission['label'] = y_pred_test

# Guardar el archivo final
filename = f"regresion_logistica_{datetime.datetime.now().strftime('%Y%m%d')}.csv"
submission.to_csv(filename, index=False)

print("¡Archivo 'submission.csv' generado exitosamente!")


¡Archivo 'submission.csv' generado exitosamente!
